# Najprostszy z możliwych model n-gramowy

In [1]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta

importlib.reload(funcs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\praca\\funcs.py'>

In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

In [5]:
%%time
n=1
words, words_model = list(), dict()
for speech in sp_joined['speech_raw']:
    words = list()
    words.extend(speech.split())
    for i in range(len(words)-n):
        speech_part = ' '.join(words[i:i+n])
        if speech_part not in words_model.keys():
            words_model[speech_part]=[]
        words_model[speech_part].append(words[i+n])
        
#Wall time: 3min 30s        

Wall time: 2min 14s


In [6]:
model1 = words_model

In [4]:
model2 = words_model

In [3]:
file = open('00_dane/model1', 'rb')

# dump information to that file
words_model = pickle.load(file)

# close the file
file.close()

In [30]:
def spin_doctor(speech_start, steps, words_model, n):
    if len(speech_start)==0:
        r = random.randrange(len(words_model.keys()))
        speech_start = list(words_model.keys())[r]
        
    if speech_start in words_model.keys():
        speech = list()
        words = speech_start
        speech.append(speech_start.capitalize())
        for _ in range(steps):
            possibilities = words_model[words]
            next_item = possibilities[random.randrange(len(possibilities))]
            speech.append(next_item)
            words = ' '.join(speech[-n:])
            words = ' '.join(words.split()[-n:])
            
        return ' '.join(speech)+'.'
    else:
        print('Brak w modelu')

Output modelu na 1-gram

In [18]:
print(spin_doctor('', 60, model1, 1))

Urobek podkomisji, mają stosować się zadania i tylko chęci ani upraw na czym operatorzy i przytułków dla polskiej żeglugi śródlądowej do Prawa Podatkowego, która zwalnia urzędników. Muszę zresztą widać w którym funkcjonuje już wyżej wymienionym projektem ustawy o których niesławnymi bohaterami narodowymi funduszami spójności terminologicznej. Wydaje się, że nasilają się zgodnie z mechanizmów, które toczą się teraz tego demo potrafi przeprowadzić.


In [22]:
print(spin_doctor('Dziś jest', 100, model2, 2))

Dziś jest już ustalony system finansów publicznych? Czy może mi pan minister mówi, że ta krytyka zasługuje na aprobatę argument zakładający, iż prezes Izby Cywilnej oraz Izby Kontroli w dniu 2 marca 2006 r. o świadczeniach przedemerytalnych też jest możliwe. Zdaniem prezydenta narusza to uprawnień przedsiębiorcy, naprawienie szkody wyrządzonej czynem niedozwolonym oraz pracowników cywilnych. Dla obydwu grup wyważyć. Pamiętajmy także o zwykłą większość głosów. Co to oznacza? Jest to bardzo ważna kwestia, to mianowicie, że w tej sytuacji pogłębia fakt, że pomoc państwa dla firm oraz poparcie dla tego rodzaju działalności tak specyficznych instytucji, jakimi są Policja, ośrodki pomocy społecznej regulują oczywiście.


In [35]:
print(spin_doctor('', 100, model2, 2))

Złudzeń, jakie są koszty badania i analizy odnoszące się do dynamiki eksportu nad importem alkoholi. Takim skutecznym i wystarczającym do uruchomienia Krajowego Centrum Hodowli Zwierząt na temat której dzisiaj mówimy o odsetkach wynoszących 12%, ale nie jest ustawą zasadniczą, którą postulowało rozwiązanie samorządowe. Czy ministerstwa podejmą działania na szkodę gospodarki, obywateli. Natomiast nie akceptujemy takich zachowań. Nawet jeśli jest ono słuszne. Aby się o swoje doświadczenia w zarządzaniu oświatą. W związku z tym, że księga wieczysta jest rejestrem centralnym, w budżecie na wydatki zwiazane z tą ostatnią, muszę powiedzieć, że to nie jest możliwe do zsumowania dopiero po 1 maja 2004 r..


## Spróbujmy ulepszyć model lepiej uwzględniając początek i koniec zdania.

In [15]:
dane_do_modelu = ' '.join(sp_joined['speech_raw'])

Usuńmy kropkę po "r.", przecinki, wielokrotne spacje.

In [16]:
dane_do_modelu = re.sub('\s+',' ',dane_do_modelu)
dane_do_modelu = re.sub('\ r.',' r',dane_do_modelu)

In [21]:
dane_do_modelu = re.sub('[!?.] ',' </s> <s> ',dane_do_modelu.lower())

In [23]:
dane_do_modelu = re.sub(',','',dane_do_modelu)

### Budowa modelu 1-gramowego

In [26]:
%%time
model1 = model_ng(dane_do_modelu, 2)

Wall time: 3min 26s


In [33]:
kl=list(model1.keys())

In [37]:
kl[100:120]

['w stanie',
 'stanie wojennym',
 'wojennym byli',
 'byli skazywani',
 'skazywani przez',
 'przez niekiedy',
 'niekiedy psychopatycznych',
 'psychopatycznych sędziów',
 'sędziów na',
 'na wyroki',
 'wyroki które',
 'które nie',
 'nie mają',
 'mają nic',
 'nic wspólnego',
 'wspólnego z',
 'z wymiarem',
 'wymiarem sprawiedliwości',
 'sprawiedliwości </s>',
 '<s> ale']